# API Deployment

The idea is to run this scraper on an API to have it nice and handy. The scraper code needs to be adapted to Flask, and that is what we are going to do in this jupyter notebook. We will understand how it will work a file that we will call **app.py**.

That file is a Flask app created to deploy an API program based on Python. With that being said, let's go!

# Flask & HTML

Flask is a Microframework web to dev some simple apps. It could be used in this case to create a simple web page that will serve us as method to deploy the python script **scraper.py** in a webpage in a simple and portable way.

Besides, it's quite simple to understand and learn how to use it. So in this step, we will understand how to create and deploy a web page based on Flask and using a *html template*

In [2]:
from flask import Flask

app = Flask(__name__)

# where is going to be executes the app. A server, in this case, our local machine

@app.route('/')

def hello_world():
    return 'Hello world'

if __name__ == '__name__':
    app.run(debug =True)

Last cell code is the general syntax to an Flask App. We instance an app object from Flask module. This is like the gorund where we will build out wonderfull app.

Now, what can we do with it? First, we will need to understand a couple of key concepts.

1) Routes: It defines the URL that the app could handle and which python function will execute when accesing this url

2) HTTP methods: These protocols like GET and POST are the "common tongue" between a web browser and servers. A simple way to see it is that, we use "GET" to get data from a server and "POST" to send it.

    In Flask, the object request acces to the data of the request that the user is sending. In this case, through the web it would send an GET or POST method. 

3) THe index function will be executed when the user try to acces to de route 

With that in mind, the web page need a formu to be filled with the game that the user is intersted in getting info, and behind all that, the program will execute the scraper. So, our first step is recieving info from the web page.

This implies that the first method to be constructed is a POST method from the user. Now, we can just ommit the part in *scraper.py* where  we create a function to transform the input from the user in a propper URL.

Now, we should star by developing a simple idea of how the webpage should be looking like. But let's not worry for it rigth now. 

In [4]:
# Here, we are designing allowed methods:
def index():
    if request.method == 'POST':
        # request.form: a dictionary with the info sent it by a html form
        # request.form.getlist('games': get alist of values associated with tag games
        items = request.form.getlist('games')
        # We requiere to construc the url within the user input on the webpage, list comprehension to do it
        us_urls = [f"https://psdeals.net/us-store/search?search_query={i.replace(' ', '+')}" for i in items]
        tr_urls = [f"https://psdeals.net/tr-store/search?search_query={i.replace(' ', '+')}" for i in items]
        # and we use the function that we already created
        us_games_info = fetch_game_info(us_urls, items, 'USD')
        tr_games_info = fetch_game_info(tr_urls, items, 'TRY')
        
        # this is the same idea that we already saw in scarper.py
        for game, details in tr_games_info.items():
            for detail in details:
                for us_detail in us_games_info.get(game, []):
                    if detail['Platform'] == us_detail['Platform']:
                        detail['Price (USD)'] = us_detail['Price (USD)']

        data = []
        for game, details in tr_games_info.items():
            for detail in details:
                detail['Game'] = game
                data.append(detail)
                
        df = pd.json_normalize(data)

        url = "https://www.google.com/search?q=lira+turca+a+dolar&oq=lira+turca+a+dolar&gs_lcrp=EgZjaHJvbWUqBggAEEUYOzIGCAAQRRg7MgYIARAuGEDSAQgyMzU4ajBqMagCALACAA&sourceid=chrome&ie=UTF-8"
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept-Language': 'en-US,en;q=0.9',
            'Accept-Encoding': 'identity',
            'Connection': 'keep-alive'
        }
        r = requests.get(url, headers=headers)
        soup = BeautifulSoup(r.text, "lxml")
        requested_price = soup.find_all("span", class_="DFlfde SwHCTb")
        dolar_tr = [price['data-value'] for price in requested_price]
        dolar_tr = float(dolar_tr[0])
        print("A dolar is equal to {} TRY".format(dolar_tr))

        df["Price (TRY)"] = df["Price (TRY)"].str.replace("TL", "")
        df["Price (USD)"] = df["Price (USD)"].str.replace("FREE", "0")
        df["Price (TRY)"] = df["Price (TRY)"].str.replace("FREE", "0")
        df["Price (TRY)"] = df["Price (TRY)"].str.replace(",", "")
        df["Price (USD)"] = df["Price (USD)"].str.replace("$", "")

        df["Price (USD)"] = df["Price (USD)"].astype(float)
        df["Price (TRY)"] = df["Price (TRY)"].astype(float)
        df["Price (TRY - USD)"] = round(df["Price (TRY)"] * dolar_tr, 2)
        df["Difference (US tore- TR store)"] = round(df["Price (USD)"] - df["Price (TRY)"] * dolar_tr, 2)
        df = df.iloc[:, [0, 1, 3, 5, 6, 2]]
        # A final return: renders the data scraped in a html webpage
        return render_template('index.html', tables=[df.to_html(classes='data')], titles=df.columns.values)
    return render_template('index.html')



# Template

Alongside the API, a template should be generated on which the API will run to display the information in a browser.

A folder with the template for this browser is built.

Once this is created, a final file is added to allow the execution of the render.

The procfile is created with this command for the console in dev.

To this task, let us use ChatGPT to create a abse template. The output from chat gpt promt was already stored in this folder as html_dev.md with a short explenation of its structure. 

In this part, i would like to focus on a better web page structure, but to give you a general idea, you could work with the structure of the html_dev.md file which stored the concept of what we want.

But what really matters is that is working. We could now add some styling and context, colors and more with Chat GPT's help. But for now, we are done her. 

# What's next?

These simple steps.

On the dev folder, create the app.py file to sotre the Flask app, the html file modified and the template, yaml and so on.